In [30]:
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/samraatsapehia/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [31]:
import numpy
import tflearn
import tensorflow
import random
import json
import pickle
from gtts import gTTS
import os
language = 'en'

with open("finalintent.json") as file:
	data = json.load(file)

Instructions for updating:
non-resource variables are not supported in the long term


In [32]:

words = []
labels = []
docs_x = []
docs_y = []

for intent in data["intents"]:
	for pattern in intent["patterns"]:
		wrds = nltk.word_tokenize(pattern)
		words.extend(wrds)
		docs_x.append(wrds)
		docs_y.append(intent["tag"])

	if intent["tag"] not in labels:
		labels.append(intent["tag"])

words = [stemmer.stem(w.lower()) for w in words if w != "?"]
words = sorted(list(set(words)))

labels = sorted(labels)


training = []
output = []

out_empty = [0 for _ in range(len(labels))]

for x, doc in enumerate(docs_x):
	bag = []

	wrds = [stemmer.stem(w) for w in doc]

	for w in words:
		if w in wrds:
			bag.append(1)
		else:
			bag.append(0)
	output_row = out_empty[:]
	output_row[labels.index(docs_y[x])] = 1

	training.append(bag)
	output.append(output_row)

training = numpy.array(training)
output = numpy.array(output)


In [33]:
tensorflow.compat.v1.reset_default_graph()

net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


2024-05-17 02:22:19.325415: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Max
2024-05-17 02:22:19.325437: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 48.00 GB
2024-05-17 02:22:19.325451: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 18.00 GB
2024-05-17 02:22:19.325471: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-05-17 02:22:19.325485: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2024-05-17 02:22:19.390355: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled
2024-05-17 02:22:19.392487: I tensorflow/core/gr

In [34]:
model.load("model.tflearn")


INFO:tensorflow:Restoring parameters from /Users/samraatsapehia/tft/model.tflearn


2024-05-17 02:22:23.466083: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-05-17 02:22:23.466104: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2024-05-17 02:22:23.515155: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-05-17 02:22:23.515174: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL,

In [35]:
def bag_of_words(s,words):
	bag = [0 for _ in range(len(words))]


	s_words = nltk.word_tokenize(s)
	s_words = [stemmer.stem(word.lower()) for word in s_words]

	for se in s_words:
		for i, w in enumerate(words):
			if w == se:
				bag[i] = 1

	return numpy.array(bag)

In [51]:
def chat():
    print("Start Talking with the bot, type quit to stop!")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break

        results = model.predict([bag_of_words(inp, words)])[0]
        results_index = numpy.argmax(results)
        tag = labels[results_index]

        if results[results_index] > 0.5:
            for tg in data["intents"]:
                if tg['tag'] == tag:
                    responses = tg['responses']
                    chatvoice=random.choice(responses)
                    mytext = chatvoice
                    print(chatvoice)
                    myobj = gTTS(text=mytext, lang=language, slow=False)
                    myobj.save("welcome.mp3")
                    os.system("afplay welcome.mp3")
            print("\n")
        else:
            print("I didnt get that, try again")


In [52]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import random
import pyttsx3
from gtts import gTTS
import os
language = 'en'



train_data = pd.read_csv('Training.csv')
X_train = train_data.drop('prognosis', axis=1)
y_train = train_data['prognosis']


model1 = RandomForestClassifier()
model1.fit(X_train, y_train)

def collect_symptoms():
    symvoice="Enter your symptoms separated by commas:"
    myobj = gTTS(text=symvoice, lang=language, slow=False)
    myobj.save("symoice.mp3")
    os.system("afplay symvoice.mp3")
    print("Enter your symptoms separated by commas:")
    symptoms_input = input("Symptoms: ").lower().strip()
    symptoms_list = symptoms_input.split(',')
    symptoms_vector = np.zeros(len(X_train.columns))
    for symptom in symptoms_list:
        symptom = symptom.strip()
        if symptom in X_train.columns:
            symptoms_vector[X_train.columns.get_loc(symptom)] = 1
    return symptoms_vector


def main():
    cvoice="Do you want 'diagnosis' or 'first aid' help? Type your choice: "
    myobj = gTTS(text=cvoice, lang=language, slow=False)
    myobj.save("cvoice.mp3")
    os.system("afplay cvoice.mp3")
    choice = input("Do you want 'diagnosis' or 'first aid' help? Type your choice: ").lower().strip()
    if choice == 'diagnosis':
        symptoms_vector = collect_symptoms()
        prognosis = model1.predict([symptoms_vector])[0]
        diagvoice=f"Based on your symptoms, the diagnosis is: {prognosis},Please enter prognosis in chat below."
        mytext = diagvoice
        print(diagvoice)
        myobj = gTTS(text=mytext, lang=language, slow=False)
        myobj.save("welcome.mp3")
        os.system("afplay welcome.mp3")
        chat()




    elif choice == 'first aid':
        while True:
            chat()
    else:
        errorvoice="Invalid choice, please type 'diagnosis' or 'first aid'."
        print("Invalid choice, please type 'diagnosis' or 'first aid'.")
        myobj = gTTS(text=errorvoice, lang=language, slow=False)
        myobj.save("errorvoice.mp3")
        os.system("afplay errorvoice.mp3")

if __name__ == "__main__":
    main()


Do you want 'diagnosis' or 'first aid' help? Type your choice:  diagnosis


Error: AudioFileOpen failed ('wht?')


Enter your symptoms separated by commas:


Symptoms:  blister,red_sore_around_nose,yellow_crust_ooze


/Users/samraatsapehia/miniforge3/envs/tft1/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


...

You:  arthritis


Treatment may include NSAIDs, DMARDs, and physical therapy to manage symptoms. Consult a healthcare provider for personalized treatment.




You:  cold


1)Keeping hydrated is absolutely vital to help 'flush' out the cold, as well as to break down congestion and keep your throat lubricated. 2)Vitamin C is extremely helpful when fighting infection, so at the first sign of a cold be sure to increase your intake by eating plenty of berries, citrus fruits, papayas, broccoli and red peppers which will help keep you protected. 3)When it comes to combating a cold,Vitamin D is essential in helping to regulate immune response.




You:  quit
